In [6]:
from openai import OpenAI
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import re
import pandas as pd
import numpy as np

# 加载环境变量
load_dotenv()

# 辅助函数：根据术语库查找翻译
import re
import pandas as pd

def find_translations(input_text, original_language, target_language, glossary_df):
    # 如果 original_language 或 target_language 不是列名，则直接返回空列表
    if original_language not in glossary_df.columns or target_language not in glossary_df.columns:
        return []
    
    # 过滤掉原始术语为空的行，使用向量化方法构建字典
    valid_entries = glossary_df[glossary_df[original_language].notna()]
    term_dict = dict(zip(valid_entries[original_language], valid_entries[target_language]))
    
    # 构建正则表达式，利用 re.escape 对每个术语转义，再用 "|" 拼接起来
    # 这样可以一次性匹配所有术语
    pattern = re.compile("|".join(map(re.escape, term_dict.keys())))
    
    # 使用正则表达式查找所有匹配项，结果可能包含重复匹配
    found_terms = set(pattern.findall(input_text))
    
    # 根据匹配到的术语，构造结果列表
    results = [(term, term_dict[term]) for term in found_terms]
    return results


# 辅助函数：检测文本中是否包含特殊字符串
def contains_special_string(sentence):
    patterns = {
        "<% ... %>": r"<%.*?%>",
        "%s": r"%s",
        "{0}, {1}, {2} 等": r"{\d+}",
        "%d": r"%d",
        "{counts}": r"{counts}",
        "&{...}&": r"&{.*?}&",
        "{}": r"{}",
        "#...#": r"#.*?#",
        "{{...}}": r"{{.*?}}",
        "连续的大写英文字母（AR, AP, SKU）": r"[A-Z]{2,}",
        "大驼峰命名的单词（如 ServiceCode, LocStudio）": r"(?:[A-Z][a-z]+){2,}",
        "包含 http:// 的字符串": r"http://",
        "包含 https:// 的字符串": r"https://",
        "包含 E:\\, D:\\, C:\\ 的字符串": r"[CDE]:\\",
        "包含 datediff(.*?,.*?,.*?) 的字符串": r"datediff\(.*?,.*?,.*?\)",
        "@业务函数. ... 的字符串@": r"@业务函数\..*?@",
        "小驼峰命名的单词（如 serviceCode, locStudio）": r"[a-z]+[a-z]*[A-Z][a-zA-Z]*"
    }

    reasons = []
    matched_strings = []
    for reason, pattern in patterns.items():
        matches = re.findall(pattern, sentence)
        if matches:
            reasons.append(reason)
            matched_strings.extend(matches)
    return {
        "contains_special_string": bool(reasons),
        "reason": reasons,
        "matched_strings": matched_strings
    }

# 定义 Model 类
class Model():
    def __init__(self, modelname, selected_lora_model, selected_gpu, glossary_df):
        self.client = OpenAI()
        self.glossary_df = glossary_df

    def translate_section(self, input_text, original_language, target_languages):
        res = []
        for target_language in target_languages:
            # 特殊情况：若文本以特定标识开头、或者文本不含中英文字符、或者文本为特定标记，则直接返回原文
            if input_text.strip().startswith("[ref1]") or \
               not re.search(r'[A-Za-z\u4e00-\u9fff]', input_text.strip()) or \
               input_text.strip() in ["此词条确认无需翻译或已废弃", "!!!!!!!!", "Obsolete", "obsolete"]:
                res.append({
                    "target_language": target_language,
                    "generated_translation": input_text,
                    "geo_mean_confidence": 1
                })
            else:
                # 提取并暂时移除 markdown 中的图片（base64格式）
                removed_images = re.findall(r"!\[.*?\]\(data:image\/[^;]+;base64,[^)]+\)", input_text)
                input_text_clean = re.sub(r"!\[.*?\]\(data:image\/[^;]+;base64,[^)]+\)", "", input_text)

                # 查找术语翻译
                matches = find_translations(input_text_clean, original_language, target_language, self.glossary_df)
                if matches:
                    terminology_guide = "\n".join([f"- {item1}: {item2}" for item1, item2 in matches])
                    system_prompt = f"""
You are an expert in translating {original_language} to {target_language} for ERP systems. Your task is to translate markdown-formatted text from {original_language} to {target_language}.

Here is a terminology guide to help you ensure accurate translations for common ERP terms:
{terminology_guide}

The text to be translated may not necessarily be complete phrases or sentences, but you must translate it into the corresponding language based on your understanding while preserving its formatting.
"""
                else:
                    system_prompt = f"""
You are an expert in translating {original_language} to {target_language} for ERP systems. Your task is to translate markdown-formatted text from {original_language} to {target_language}.

The text to be translated may not necessarily be complete phrases or sentences, but you must translate it into the corresponding language based on your understanding while preserving its formatting.
"""

                messages = [{"role": "system", "content": system_prompt}]
                special_string_list = []
                for i in range(2):
                    if i == 0:
                        messages.append({"role": "user", "content": input_text_clean})
                    else:
                        messages.append({
                            "role": "user",
                            "content": f"You should skip the words: {', '.join(special_string_list)}. Do not translate these words. Please translate again without adding extra content."
                        })

                    completion = self.client.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=messages,
                        temperature=0,
                        logprobs=True,
                        top_p=1
                    )
                    translated_text = completion.choices[0].message.content
                    logprobs = [token.logprob for token in completion.choices[0].logprobs.content]
                    probs = np.exp(logprobs)
                    geo_mean_confidence = float(np.prod(probs) ** (1 / len(probs)))
                    messages.append({"role": "assistant", "content": translated_text})

                    # 检查错误信息
                    error_messages = [
                        "Sorry, I can't assist with that request",
                        "It seems like your message is incomplete"
                    ]
                    if any(error_msg in translated_text for error_msg in error_messages):
                        continue

                    temp = contains_special_string(input_text_clean)
                    if temp["contains_special_string"]:
                        all_special_strings_retained = True
                        for matched_string in temp["matched_strings"]:
                            if matched_string not in translated_text:
                                all_special_strings_retained = False
                                if matched_string not in special_string_list:
                                    special_string_list.append(matched_string)
                        if all_special_strings_retained:
                            break
                    else:
                        break

                if removed_images:
                    translated_text += "\n" + "\n".join(removed_images)

                translated_text = re.sub(r'[\u4e00-\u9fff]', '', translated_text)
                res.append({
                    "target_language": target_language,
                    "generated_translation": translated_text,
                    "geo_mean_confidence": geo_mean_confidence
                })
        return res

    # 使用多线程批量翻译多个文本，同时显示进度条
    def generate(self, inputs, original_language, target_languages, max_workers=2000):
        res = [None] * len(inputs)
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(self.translate_section, inputs[i], original_language, target_languages): i for i in range(len(inputs))}
            # 用 tqdm 包裹 as_completed 迭代器，显示进度
            for future in tqdm(as_completed(futures), total=len(futures), desc="Translating"):
                index = futures[future]
                try:
                    res[index] = future.result()
                except Exception as e:
                    res[index] = [{"target_language": target_language, "generated_translation": f"Error: {e}", "geo_mean_confidence": 0} 
                                  for target_language in target_languages]
        return res

if __name__ == "__main__":
    # 参数设置
    original_col = "简体中文(源)"  # 待翻译文本所在列
    target_col = "待翻译(译)"      # 翻译后文本存放列
    conf_col = "备注"            # 存放置信度的列
    original_language = "Chinese"
    target_language = "Thai"

    # 读取术语库
    glossary_file = r"glossary.xlsx"
    glossary_df = pd.read_excel(glossary_file)

    # 实例化 Model 对象
    model = Model("gpt-4o-mini", selected_lora_model=None, selected_gpu=None, glossary_df=glossary_df)

    # 循环处理 part_3 到 part_11 的文件
    for part in range(3, 12):
        input_excel = f"D:\\Projects\\ai-translator\\src\\multilangInitData20250210空\\part_{part}.xlsx"
        output_excel = f"D:\\Projects\\ai-translator\\src\\multilangInitData20250210空\\part_{part}_translated.xlsx"
        df = pd.read_excel(input_excel)
        inputs = df[original_col].astype(str).tolist()
        translation_results = model.generate(inputs, original_language, [target_language], max_workers=200)
        
        translated_texts = []
        confidences = []
        for result in translation_results:
            if result and isinstance(result, list) and len(result) > 0:
                d = result[0]
                translated_texts.append(d.get("generated_translation", ""))
                confidences.append(d.get("geo_mean_confidence", 0))
            else:
                translated_texts.append("")
                confidences.append(0)
        
        df[target_col] = translated_texts
        df[conf_col] = confidences
        df.to_excel(output_excel, index=False)
        print(f"翻译结果已保存到 {output_excel}")


Translating: 100%|██████████| 100000/100000 [1:33:06<00:00, 17.90it/s]     


翻译结果已保存到 D:\Projects\ai-translator\src\multilangInitData20250210空\part_3_translated.xlsx


Translating: 100%|██████████| 100000/100000 [26:43<00:00, 62.36it/s] 


翻译结果已保存到 D:\Projects\ai-translator\src\multilangInitData20250210空\part_4_translated.xlsx


Translating: 100%|██████████| 100000/100000 [25:51<00:00, 64.47it/s] 


...

翻译结果已保存到 D:\Projects\ai-translator\src\multilangInitData20250210空\part_9_translated.xlsx


Translating: 100%|██████████| 100000/100000 [29:30<00:00, 56.48it/s] 


翻译结果已保存到 D:\Projects\ai-translator\src\multilangInitData20250210空\part_10_translated.xlsx


Translating: 100%|██████████| 30626/30626 [16:38<00:00, 30.68it/s] 


翻译结果已保存到 D:\Projects\ai-translator\src\multilangInitData20250210空\part_11_translated.xlsx


In [4]:
import pandas as pd
import re

# 定义文件路径
input_file = r"D:\Projects\ai-translator\src\multilangInitData20250210空\part_1_translated.xlsx"
output_file = r"D:\Projects\ai-translator\src\multilangInitData20250210空\part_1_translated.xlsx"

# 读取 Excel 文件
df = pd.read_excel(input_file)

# 删除 "待翻译(译)" 列中所有中文字符（Unicode 范围：\u4e00 - \u9fff）
df["待翻译(译)"] = df["待翻译(译)"].astype(str).apply(lambda x: re.sub(r'[\u4e00-\u9fff]', '', x))

# 保存结果到新的 Excel 文件
df.to_excel(output_file, index=False)

print(f"处理完成，结果已保存到 {output_file}")


处理完成，结果已保存到 D:\Projects\ai-translator\src\multilangInitData20250210空\part_1_translated.xlsx
